In [ ]:
import matplotlib
matplotlib.use('pgf')
matplotlib.rc('pgf', texsystem='pdflatex')  # from running latex -v
preamble = matplotlib.rcParams.setdefault('pgf.preamble', [r'\usepackage{color}'])
# preamble.append(r'\usepackage{color}')
 
# import matplotlib.pyplot as plt
# plt.rc('text', usetex=True)
# plt.rc('text.latex', preamble=r'\usepackage{color}')
# matplotlib.verbose.level = 'debug-annoying'


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.transforms import Affine2D, ScaledTranslation

# plt.rcParams.update({'font.size': 22})

In [ ]:
df_2_class = pd.read_csv('../data/2_class_scores.csv')
df_2_class.head(7)

In [ ]:
df_3_class = pd.read_csv('../data/3_class_scores.csv')
df_3_class.head(7)

In [ ]:
f1_key = 'F1-test-all-mod'
f1_ci_key = 'F1-CI-test-all-mod'
mcc_key = 'MCC-test-all-mod'
mcc_ci_key = 'MCC-CI-test-all-mod'

In [ ]:
cmap_cont = LinearSegmentedColormap.from_list('mycmap', ['#b0cffb', '#22418e'])
cmap_disc = ListedColormap(['#b0cffb', '#7f96cf','#22418e'])
cmap_disc_rev = ListedColormap(['#7f96cf','#b0cffb'])
cmap_disc_light =  ListedColormap(['#b0cffb'])
cmap_disc_middle = ListedColormap(['#7f96cf'])
cmap_disc_dark = ListedColormap(['#22418e'])

In [ ]:
def limit_err_values(df, metric: str, metric_ci: str, eps=0.001):
    y_err = np.zeros((2, len(df[metric_ci])))
    for i in range(len(df[metric_ci])):
        if df[metric][i] + df[metric_ci][i] > 1 - eps:  # eps to avoid clipping
            y_err[1, i] = 1 - df[metric][i] - eps  # eps to avoid clipping
        else:
            y_err[1, i] = df[metric_ci][i]
        if df[metric][i] - df[metric_ci][i] < 0 + eps:
            y_err[0, i] = df[metric][i] - eps  # eps to avoid clipping
        else:
            y_err[0, i] = df[metric_ci][i]
    return y_err


In [ ]:
def plot_bar(df):
    fig, ax = plt.subplots(figsize=(10, 6))

    kwargs = {
        'kind': 'bar',
        'x': 'model',
        'ylim': (0, 1.0),
        'ax': ax,
        'width': 0.3,
        'capsize': 2,
        'ecolor': 'black',
    }

    y_err_f1 = limit_err_values(df, f1_key, f1_ci_key)
    y_err_mcc = limit_err_values(df, mcc_key, mcc_ci_key)
    
    df.plot(y=f1_key, yerr=y_err_f1, colormap=cmap_disc_dark, position=1, **kwargs)
    # df.plot(y='F1-test', yerr='CI-F1-test', colormap=cmap_disc_middle, position=1, **kwargs)
    df.plot(y=mcc_key, yerr=y_err_mcc, colormap=cmap_disc_middle, position=0, **kwargs)

    # df.plot(kind='bar', x='model', y='F1-test', yerr='CI-F1-test', ylim=(0, 1.0), colormap=cmap_disc_middle, ax=ax, position=1, width=width, capsize=2)
    # df.plot(kind='bar', x='model', y='MCC-test', yerr='CI-MCC-test', ylim=(0, 1.0), colormap=cmap_disc_light, ax=ax, position=0, width=width)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Score')
    plt.xlabel('Modalities')

    plt.xticks(list(range(7)), [r'$\textcolor{blue}{PET}$', 'MRI', 'Tabular', 'PET-MRI', 'PET-Tabular', 'MRI-Tabular', 'All modalities'])

    ax.set_position([0.1, 0.1, 0.6, 0.8])
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.9), frameon=False, labels=['F1', 'MCC'])
    plt.xlim(-0.55, None)

    plt.vlines([2.5, 5.5], ymin=0, ymax=1, color='black', linestyles='dashed')

    ax.spines[['right', 'top']].set_visible(False)
    return fig


In [ ]:
fig = plot_bar(df_2_class)

In [ ]:
fig.savefig('../data/images/2_class_scores_bars.png', dpi=300, transparent=True, bbox_inches='tight', backend='pgf')

In [ ]:
fig = plot_bar(df_3_class)
fig.savefig('../data/images/3_class_scores_bars.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:


def plot_val_bar(df):
    fig, ax = plt.subplots(figsize=(10, 6))

    kwargs = {
        'kind': 'bar',
        'x': 'model',
        'ylim': (0, 1.0),
        'ax': ax,
        'width': 0.3,
        'capsize': 2,
        'ecolor': 'black',
    }
    
    df.plot(y='val-F1', colormap=cmap_disc_middle, **kwargs)
    # df.plot(y='F1-test', yerr='CI-F1-test', colormap=cmap_disc_middle, position=1, **kwargs)
    # df.plot(y='MCC', yerr=y_err_mcc, colormap=cmap_disc_light, position=0, **kwargs)

    # df.plot(kind='bar', x='model', y='F1-test', yerr='CI-F1-test', ylim=(0, 1.0), colormap=cmap_disc_middle, ax=ax, position=1, width=width, capsize=2)
    # df.plot(kind='bar', x='model', y='MCC-test', yerr='CI-MCC-test', ylim=(0, 1.0), colormap=cmap_disc_light, ax=ax, position=0, width=width)
    plt.xticks(rotation=45)
    plt.ylabel('Score')
    plt.xlabel('Modalities')

    ax.set_position([0.1, 0.1, 0.6, 0.8])
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.9), frameon=False, labels=['F1', 'MCC'])
    plt.xlim(-0.55, None)

    ax.spines[['right', 'top']].set_visible(False)
    return fig

In [ ]:
fig = plot_val_bar(df_2_class)
fig.savefig('../data/images/2_class_val_scores_bars.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
fig = plot_val_bar(df_3_class)
fig.savefig('../data/images/3_class_val_scores_bars.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
def plot_points(df):
    fig, ax = plt.subplots()
    trans1 = Affine2D().translate(-0.1, 0.0) + ax.transData
    trans2 = Affine2D().translate(+0.1, 0.0) + ax.transData

    y_err_f1 = limit_err_values(df, f1_key, f1_ci_key, eps=0.002)
    y_err_mcc = limit_err_values(df, mcc_key, mcc_ci_key, eps=0.002)

    plt.errorbar(x=df['model'], y=df[f1_key], yerr=y_err_f1, fmt='o', color='#22418e', transform=trans1, capsize=2, label='F1')
    plt.errorbar(x=df['model'], y=df[mcc_key], yerr=y_err_mcc, fmt='o', color='#7f96cf', transform=trans2, capsize=2, label='MCC')

    plt.xticks(rotation=45)
    plt.ylabel('Score')
    plt.xlabel('Modalities')
    plt.legend(['F1', 'MCC'])

    plt.ylim(0, 1.0)
    ax.spines[['right', 'top']].set_visible(False)

    # position legend in lower right corner
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.9))
    return fig

In [ ]:
fig = plot_points(df_2_class)
fig.savefig('../data/images/2_class_scores_points.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
fig = plot_points(df_3_class)
fig.savefig('../data/images/3_class_scores_points.png', dpi=300, transparent=True, bbox_inches='tight')